## Importation librairie et donnée


In [120]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import re
import os
import time
import matplotlib.pyplot as plt

In [121]:
# Chemin du dossier dans systeme
chemin_dossier_pln_janvier = '../data/stan_janvier_2024'
chemin_dossier_pln_mars = '../data/stan_mars_2024'
chemin_dossier_abacus = '../data/ABACUS'
chemin_dossier_table_systeme = '../data/TABLE_SYSTEME'
chemin_dossier_table_fusion = '../data/fusion_radar'

# Liste des fichiers dans le dossier
contenu_dossier_pln_janvier = os.listdir(chemin_dossier_pln_janvier)
contenu_dossier_pln_mars = os.listdir(chemin_dossier_pln_mars)
contenu_dossier_abacus = os.listdir(chemin_dossier_abacus)
contenu_dossier_table_systeme = os.listdir(chemin_dossier_table_systeme)
contenu_dossier_table_fusion = os.listdir(chemin_dossier_table_fusion)

In [122]:
pd.set_option('display.max_columns', 150)

In [123]:
def read_and_process_file(fichier_a_deposee):
    vol_prevu = []
    vol_fini = []
    vol_termine = []
    tableau_vol = {}
    tableaux_vol = []
    iter = 0
    flag82 = False
    hneg = False
    compt82 = 0
    num81 = 0
    prevu = False
    termine = False
    final = False
    complet = 0
    output = pd.DataFrame()
    compteur = 0
    tableau_vol = {}
    compteurCcr = 0

    with open(fichier_a_deposee, 'r') as fichier:
        for i, ligne in enumerate(fichier):
            words = ligne.split()
            if words[0] == "02":
                date_str = words[1]
                date_obj = datetime.strptime(date_str, "%d-%m-%Y")
                date_fichier = date_obj.timetuple().tm_yday
            if words[0] == "05":
                if tableau_vol:
                    tableau_vol["isPrevu"] = isprevu
                    tableau_vol["isRealise"] = isrealise
                    tableau_vol["isFinal"] = isfinal
                    df_dictionary = pd.DataFrame([tableau_vol])
                    output = pd.concat([output, df_dictionary], ignore_index=True)
                tableau_vol = {}
                isprevu = False
                isrealise = False
                isfinal = False
                is81 = False
                is82 = False
                balise = ''
                hneg = False
                flag82 = False
                etat = ''
            if words[0] == "11":
                etat = 'prevu'
                isprevu = True
            if words[0] == "12":
                etat = 'realise'
                isrealise = True
            if words[0] == "13":
                if len(words) > 1 and words[1] == "=":
                    for key in list(tableau_vol.keys()):
                        if '_prevu' in key:
                            tableau_vol[key.replace('_prevu', '_final')] = tableau_vol[key]
                etat = 'final'
                isfinal = True
            if words[0] == "14":
                etat = 'transaction'
            if words[0] == "81":
                pass
            if words[0] == "20":
                tableau_vol['callSign_' + etat] = words[1]
                tableau_vol['dep_' + etat] = words[2]
                tableau_vol['arr_' + etat] = words[3]
                tableau_vol['numCautra_' + etat] = words[4]
                tableau_vol['dateRelative_' + etat] = words[5]
                tableau_vol['typeAvion_' + etat] = words[6]
                tableau_vol['work_' + etat] = words[7]
                if words[8][:2] == '??':
                    pass
                else:
                    tableau_vol['work1' + etat] = words[8].strip().ljust(9)
            if words[0] == "21":
                tableau_vol['heuresDep_' + etat] = words[1]
                tableau_vol['RFL_' + etat] = words[2]
                tableau_vol['vitesse_' + etat] = words[3]
                tableau_vol['EOBT_' + etat] = words[4]
            if words[0] == "22":
                tableau_vol['regleVol_' + etat] = words[1]
                tableau_vol['typeVol_' + etat] = words[2]
                tableau_vol['HeurePremiereBaliseActive_' + etat] = words[10]
                if words[3][:2] == '??':
                    pass
                else:
                    tableau_vol['IFPL_' + etat] = words[3].strip().ljust(10)
                tableau_vol['plnActive_' + etat] = words[4]
                tableau_vol['typePlnStan']= words[6]
                tableau_vol['plnAnnule_' + etat] = words[5]
                if '??' in words[7]:
                    pass
                elif len(words[7]) == 8:
                    date_str = words[7]
                    date_obj = datetime.strptime(date_str, '%d%m%Y')
                    day_vol = date_obj.timetuple().tm_yday
                    tableau_vol['dateBlock_' + etat] = words[7][:4] + words[7][6:]
                else:
                    tableau_vol['dateBlock_' + etat] = words[7].strip().ljust(6)
            if words[0] == "23":
                if "??" in words[4]:
                    tableau_vol['adresseModeS_' + etat] = np.NaN
                else:
                    tableau_vol['adresseModeS_' + etat] = words[4]
            if words[0] == "24":
                tableau_vol['numeroPLNM' + etat] = words[1]
                tableau_vol['flightID' + etat] = words[2]
            if words[0] == "31":
                tableau_vol['balise' + etat] = words[1]
            if words[0] == "32":
                tableau_vol['HeurePremiereBalise_' + etat] = words[1]
            if words[0] == "33":
                tableau_vol['listeBalises' + etat] = words[1]
            if words[0] == "36":
                tableau_vol['indicateur' + etat] = words[1]
            if words[0] == "41":
                tableau_vol['carte' + etat] = words[1]
            if words[0] == "71":
                tableau_vol['centreTraversé' + etat] = words[1:]
            if words[0] == "72":
                tableau_vol['listeRangPremier' + etat] = words[1]
            if words[0] == "80":
                tableau_vol['rangTransaction' + etat] = words[1]
            if words[0] == "81" and not is81:
                is81 = True
                if len(words) >= 15:
                    parts = ligne.split("-")
                    last_word = parts[0].split()[-1]
                    if "ABI" in ligne:
                        tableau_vol['typePln'] = "ABI"
                    if "(FPL" in parts[0] or "(CHG)" in parts[0]:
                        tableau_vol['case7'] = parts[1]
                        tableau_vol['case8'] = parts[2]
                        tableau_vol['case9'] = parts[3]
                        tableau_vol['case10'] = parts[4]
                        tableau_vol['case13'] = parts[5]
                        tableau_vol['case15'] = parts[6]
                        if len(parts) > 8:
                            tableau_vol['case16'] = parts[7]
                            tableau_vol['case18'] = parts[8]
                            if tableau_vol['case18'] == "RPL":
                                tableau_vol['typePln'] = "RPL"
                        else:
                            #print(ligne)
                            compteur += 1
                    elif "(APL" in parts[0]:
                        tableau_vol['case7'] = parts[1]
                        tableau_vol['case8'] = parts[2]
                        tableau_vol['case9'] = parts[3]
                        tableau_vol['case10'] = parts[4]
                        tableau_vol['case13'] = parts[5]
                        tableau_vol['case15'] = parts[6]
                        tableau_vol['typePln'] = "APL"
                        if len(parts) > 8:
                            tableau_vol['case16'] = parts[7]
                            tableau_vol['case18'] = parts[-1]
                        else:
                            compteur += 1
            if words[0] == "82"and not is82:
                is82 = True
                tableau_vol['heure'] = (words[1][:2])
                tableau_vol['minute'] = (words[1][3:])
                tableau_vol['accuseTrt' + etat] = words[1]
                if "CCR:" in ligne:
                    compteurCcr = 0
                    for word in words:
                        compteurCcr += 1
                        if word == "CCR:":
                            break
                    tableau_vol['ccrArrival'] = words[compteurCcr]
            if words[0] == "84":
                tableau_vol['final' + etat] = words[1]

    return output

def convert_and_calculate(df,date_obj):
    df['HeurePremiereBaliseActive_realise'] = df['HeurePremiereBaliseActive_realise'].astype('Int64')
    df['HeurePremiereBaliseActive_final'] = df['HeurePremiereBaliseActive_final'].astype('Int64')
    df['HeurePremiereBalise_final'] = df['HeurePremiereBalise_final'].astype('Int64')
    df['dateRelative_realise'] = df['dateRelative_realise'].astype('Int64')
    df['dateRelative_final'] = df['dateRelative_final'].astype('Int64')

    def calcul_HeureDeReference(row):
        try:
            if not pd.isna(row['dateRelative_realise']) and not pd.isnull(row['dateRelative_realise']):
                if not pd.isna(row['HeurePremiereBaliseActive_realise']) and int(row['HeurePremiereBaliseActive_realise']) != 0:
                    return int(row['HeurePremiereBaliseActive_realise']) + (-1440 if row['dateRelative_realise'] == -1 else 1440 if row['dateRelative_realise'] == 1 else 0)
                elif not pd.isna(row['HeurePremiereBalise_final']):
                    return int(row['HeurePremiereBalise_final']) + (-1440 if row['dateRelative_realise'] == -1 else 1440 if row['dateRelative_realise'] == 1 else 0)
            elif not pd.isna(row['dateRelative_final']):
                if not pd.isna(row['HeurePremiereBaliseActive_final']) and int(row['HeurePremiereBaliseActive_final']) != 0:
                    return int(row['HeurePremiereBaliseActive_final']) + (-1440 if row['dateRelative_final'] == -1 else 1440 if row['dateRelative_final'] == 1 else 0)
                elif not pd.isna(row['HeurePremiereBalise_final']):
                    return int(row['HeurePremiereBalise_final']) + (-1440 if row['dateRelative_final'] == -1 else 1440 if row['dateRelative_final'] == 1 else 0)
        except Exception:
            return None

    df['heure_de_reference'] = df.apply(calcul_HeureDeReference, axis=1)

    def calcul_DateDeReference(row):
        try:
            if not pd.isna(row['dateRelative_realise']) and not pd.isnull(row['dateRelative_realise']):
                if row['dateRelative_realise'] == 0:
                    return date_obj
                elif row['dateRelative_realise'] == 1:
                    return date_obj + timedelta(days=1)
                elif row['dateRelative_realise'] == -1 and int(row['heure_de_reference'])<0:
                    return date_obj - timedelta(days=1)
                else:
                    return date_obj
            elif not pd.isna(row['dateRelative_final']) and not pd.isnull(row['dateRelative_final']):
                if row['dateRelative_final'] == 0:
                    return date_obj
                elif row['dateRelative_final'] == 1:
                    return date_obj + timedelta(days=1)
                elif row['dateRelative_final'] == -1 and int(row['heure_de_reference'])<0:
                    return date_obj - timedelta(days=1)
                else:
                    return date_obj
        except Exception as e:
            return None

    df['date_de_reference'] = df.apply(calcul_DateDeReference, axis=1)
    return df

In [124]:
from datetime import datetime, timedelta

def dynamic_analysis(target_date_str):
    # Convert the target_date_str to a datetime object
    target_date = datetime.strptime(target_date_str, "%d-%m-%Y")

    # Calculate the previous and next days
    previous_day = target_date - timedelta(days=1)

    next_day = target_date + timedelta(days=1)


    # Format the dates to match the file naming convention
    previous_day_str = previous_day.strftime("%Y%m%d")
    target_date_str = target_date.strftime("%Y%m%d")
    next_day_str = next_day.strftime("%Y%m%d")

    # Construct file names
    nom_fichier_jour_1 = f'../data/stan_mars_2024/RDVC-{previous_day_str}.pln'
    nom_fichier_jour_2 = f'../data/stan_mars_2024/RDVC-{target_date_str}.pln'
    nom_fichier_jour_3 = f'../data/stan_mars_2024/RDVC-{next_day_str}.pln'

    # Read and process files
    fichier_jour_1 = read_and_process_file(nom_fichier_jour_1)
    fichier_jour_2 = read_and_process_file(nom_fichier_jour_2)
    fichier_jour_3 = read_and_process_file(nom_fichier_jour_3)

    # Convert and calculate
    date_obj = previous_day
    jour_1 = convert_and_calculate(fichier_jour_1,date_obj)

    date_obj = target_date
    jour_2 = convert_and_calculate(fichier_jour_2,date_obj)

    date_obj = next_day
    jour_3 = convert_and_calculate(fichier_jour_3,date_obj)


    # Return the results
    return jour_1, jour_2, jour_3

# Example usage
dateAnalyse='21-03-2024'
jour_1, jour_2, jour_3 = dynamic_analysis(dateAnalyse)

# Algo 2.3 et 2.4

## Tables utilisées

In [125]:
INDICATIF_FICTIF = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"indicatifsfictifs"), sep=';', skiprows=3)
EUROPE_SUD = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"payssud"), sep=';', skiprows=3)
AERONEFS_DE_MOINS_DE_2_TONNES = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aeronefsmoin2tonnes"), sep=';', skiprows=3)
INDICATEURS_D_EMPLACEMENT_FAUX = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aerodromesfaux"), sep=';', skiprows=3)
AERODROME_A_VERIFIER = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aerodromesaverifier"), sep=';', skiprows=3)
AERODROMES_D_APPROCHE = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aerodromesapproche"), sep=';', skiprows=3)
AERODROMES_FRONTALIERS = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aerodromesfrontaliers"), sep=';', skiprows=3)
INDICATIF_A_CODE_AUTO = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"indicatifscodesauto"), sep=';', skiprows=3)
INDICATIFS_A_STRUCTURE_TRIGRAMME_CORRECTE = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"structurtrigrammecorrects"), sep=';', skiprows=3)
COMPAGNIES_BIGRAMMES_ET_SUFFIXES = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"cgniebigrammesetsuffixes"), sep=';', skiprows=3)
COMPAGNIES_AVEC_TRIGRAMME = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"compagniesavectrigrammesautorise"), sep=';', skiprows=3)
INDICATIFS_TRIGRAMME_A_VERIFIER = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"structuretrigrammeverif"), sep=';', skiprows=3)
LETTRES_AIR_FRANCE = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"lettresairfrance"), sep=';', skiprows=3)
LETTRES_AIR_FRANCE = LETTRES_AIR_FRANCE[~((LETTRES_AIR_FRANCE['Lettre'].str.contains("MODIFIE", na=False)) & (LETTRES_AIR_FRANCE['Code exo/trait'].isna()) & (LETTRES_AIR_FRANCE['Commentaire'].isna()))]
LETTRES_AIR_FRANCE.reset_index(drop=True, inplace=True)
IMMATRICULATION_A_VERIFIER = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"immataverif"), sep=';', skiprows=3)
IMMATRICULATION_CORRECTE = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"immatcorrectes"), sep=';', skiprows=3)
OPERATEURS_MILITAIRES = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"oprmili"), sep=';', skiprows=3)
AERONEFS_STRICTEMENT_MILITAIRES = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"aeronefmilitaire"), sep=';', skiprows=3)

In [126]:
# Fonction pour convertir un modèle en expression régulière
def pattern_to_regex(pattern):
    # Remplacer # par .*, + par [a-zA-Z] et - par [0-9]
    regex = pattern.replace('#', '.?').replace('-', '[0-9]').replace('+', '[a-zA-Z]')
    # Ajouter les délimiteurs de début et de fin
    regex = '^' + regex + '$'
    return regex

def trouver_pattern(x, pattern_list):
    for pattern in pattern_list:
        regex = pattern_to_regex(pattern)
        if re.match(regex, x):
            return True
    return False

def renvoie_pattern(x, pattern_list):
    for pattern in pattern_list:
        regex = pattern_to_regex(pattern)
        if re.match(regex, x):
            return pattern
    return "NULL"

## Prétraitement utile/inutile

In [127]:
def nbre_centre_francais(list):
    rep = 0
    for x in list:
      if x in ['PARI', 'AIX', 'BRST', 'REIM', 'BORD']:
        rep +=1
    return rep

In [128]:
def utile_inutile(element):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return None

    call_sign_value = get_valid_value(element, ["callSign_realise", "callSign_final", "callSign_prevu"])
    PLN_active_value = get_valid_value(element, ["plnActive_realise", "plnActive_final", "plnActive_prevu"])
    PLN_annule_value = get_valid_value(element, ["plnAnnule_realise", "plnAnnule_final", "plnAnnule_prevu"])
    centre_traverse_value = get_valid_value(element, ["centreTraversérealise", "centreTraverséfinal", "centreTraverséprevu"])
    dep_value = get_valid_value(element, ["dep_realise", "dep_final", "dep_prevu"])
    arr_value = get_valid_value(element, ["arr_realise", "arr_final", "arr_prevu"])

    if trouver_pattern(call_sign_value, INDICATIF_FICTIF["Indicatif"].to_list()):
        return "FICT"
    if PLN_active_value == "1":
        return "UTI"
    elif PLN_annule_value == "1":
        return "CNL"
    elif nbre_centre_francais(centre_traverse_value) >= 2:
        return "2SLF"
    elif dep_value == arr_value:
        return "CIRC"
    elif dep_value[0] == "E" and dep_value in EUROPE_SUD["Code pays"].to_list():
        return "NORD"
    elif (dep_value[0] in EUROPE_SUD["Code pays"].to_list() or dep_value[:2] in EUROPE_SUD) and dep_value[0] == "E":
        return "SUD"
    return "UTI"


In [129]:
def traitement_utile_inutile(df):
    df['utile_inutile'] = df.apply(lambda x: utile_inutile(x), axis=1)
    return df

In [130]:
def test_utile_vol_active_non_fictif():
    utilite = result[result["callSign_prevu"]=="TRA79Y"]["utile_inutile"].iloc[0]
    assert utilite == "UTI", "utilité diff de UTI"

def test_inutile_vol_fictif_avec_moins():
    utilite = result[result["callSign_prevu"]=="RFIN65"]["utile_inutile"].iloc[0]
    assert utilite == "FICT", "utilité diff de FICT"

def test_inutile_vol_annule():
    utilite = result[result["callSign_prevu"]=="THY90F"]["utile_inutile"].iloc[0]
    assert utilite == "CNL", "utilité diff de CNL"

def test_inutile_vol_centrefr():
    utilite = result[result["callSign_prevu"]=="TRA4N"]["utile_inutile"].iloc[0]
    assert utilite == "2SLF", "utilité diff de 2SLF"

def test_inutile_vol_circulaire():
    utilite = result[result["callSign_prevu"]=="FNY5118"]["utile_inutile"].iloc[0]
    assert utilite == "CIRC", "utilité diff de CIRC"

def test_utile_vol_tout_les_test_faux():
    utilite = result[result["callSign_prevu"]=="TOM22B"]["utile_inutile"].iloc[0]
    assert utilite == "UTI", "utilité diff de UTI"

def test_realise_pln_nan():
    utilite = result[result["callSign_prevu"]=="THY90F"]["utile_inutile"].iloc[0]
    assert utilite == "CNL", "utilité diff de CNL"

def test_realise_centre_nan():
    utilite = result[result["callSign_prevu"]=="UAL71"]["utile_inutile"].iloc[0]
    assert utilite == "2SLF", "utilité diff de 2SLF"

## Traitement unitaire

## Initialisation

In [131]:
def TU_init(df_utile):
    df_utile['aeronef_de_moins_de_deux_tonnes'] = False
    df_utile['vol_a_transmettre'] = False
    df_utile['vol_approche'] = False
    df_utile['vol_interieur'] = False
    df_utile['vol_frontalier'] = False
    df_utile['vol_VFR'] = False
    df_utile["code_d_exoneration"] = 'Z'
    df_utile['code_exploitant'] = 'Z'
    df_utile['compagnie_française'] = False
    df_utile['type_d_avion_militaire'] = False
    df_utile['PLN_à_verifier_TU'] = False
    df_utile['PLN_valide'] = True
    df_utile['RAZ_des_invalidites_de_type_TU'] = False
    df_utile['invalidite_TU'] = [[] for _ in range(len(df_utile))]
    df_utile['type_d_indicatif'] = ""
    return df_utile

## Algo 1

In [132]:
def TU_1(df_utile):
    def TU_1_element(x):
        # Vérification de PLN_active
        for col in ["plnActive_realise", "plnActive_final", "plnActive_prevu"]:
            if col in x and not pd.isna(x[col]):
                if x[col] == "0":
                    x['invalidite_TU'] = x.get('invalidite_TU', []) + ["NACT"]
                break

        # Vérification de typeavion
        for col in ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"]:
            if col in x and not pd.isna(x[col]):
                if x[col] == "ZZZZ":
                    x['invalidite_TU'] = x.get('invalidite_TU', []) + ["TYPAV"]
                    x['PLN_valide'] = False
                if x[col] in AERONEFS_DE_MOINS_DE_2_TONNES["Type avion"].to_list():
                    x['aeronef_de_moins_de_deux_tonnes'] = True
                break
        return x
    df_utile = df_utile.apply(TU_1_element, axis=1)
    return df_utile

## Algo 2

In [133]:
# def TU_2(df_utile):
#     def TU_2_element(x):
#         if len(x['depfinal']) != 4:
#             x['invalidite_TU'].append("DEPAR1")
#             x['PLN_valide'] = False
#         elif x['depfinal'] in INDICATEURS_D_EMPLACEMENT_FAUX:
#             x['invalidite_TU'].append("DEPAR2")
#             x['PLN_valide'] = False
#         elif trouver_pattern(x['depfinal'], AERODROME_A_VERIFIER):
#             x['invalidite_TU'].append("DVPAR3")
#             x['PLN_valide'] = False
#         if len(x['arrfinal']) != 4:
#             x['invalidite_TU'].append("ARRIV1")
#             x['PLN_valide'] = False
#         elif x['arrfinal'] in INDICATEURS_D_EMPLACEMENT_FAUX:
#             x['invalidite_TU'].append("ARRIV2")
#             x['PLN_valide'] = False
#         elif trouver_pattern(x['arrfinal'], AERODROME_A_VERIFIER):
#             x['invalidite_TU'].append("AVRIV3")
#             x['PLN_valide'] = False
#     df_utile = df_utile.apply(TU_2_element, axis=1)
#     return df_utile

In [134]:
def TU_2(df_utile):
    def TU_2_element(x):
        # Vérification des colonnes de départ
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]):
                if len(x[col]) != 4:
                    x['invalidite_TU'].append("DEPAR1")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], INDICATEURS_D_EMPLACEMENT_FAUX["Code terrain"]):
                    x['invalidite_TU'].append("DEPAR2")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], AERODROME_A_VERIFIER["Code terrain"]):
                    x['invalidite_TU'].append("DVPAR3")
                    x['PLN_valide'] = False
            break

        # Vérification des colonnes d'arrivée
        for col in ["arr_realise", "arr_final", "arr_prevu"]:
            if col in x and not pd.isna(x[col]):
                if len(x[col]) != 4:
                    x['invalidite_TU'].append("ARRIV1")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], INDICATEURS_D_EMPLACEMENT_FAUX["Code terrain"]):
                    x['invalidite_TU'].append("ARRIV2")
                    x['PLN_valide'] = False
                elif trouver_pattern(x[col], AERODROME_A_VERIFIER["Code terrain"]):
                    x['invalidite_TU'].append("AVRIV3")
                    x['PLN_valide'] = False
            break
        return x
    df_utile = df_utile.apply(TU_2_element, axis=1)
    return df_utile

## Algo 3

In [135]:
def TU_3_bis(df_utile):
    eurocontrol = ['EB','ED','EF','EG','EH','EI','EK','EL','EN','EP','ES','EV','EY','LA','LB','LC','LD','LE','LG','LH','LI','LJ','LK','LM','LO','LP','LQ','LR','LS','LT','LU','LW','LY','LZ','UD','UG','GC']
    def TU_3_bis_element(x):
        dep_value = None
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        if dep_value == "ZZZZ":
            x['PLN_valide'] = False
            x['PLN_à_verifier_TU'] = False
            x['invalidite_TU'].extend(["DEPAR2", "TRANS1"])
        elif dep_value[:2] in eurocontrol:
            x['vol_a_transmettre'] = False
        elif x["ccrArrival"] == "ALGR":
            x['vol_a_transmettre'] = True
        elif x["ccrArrival"] == "ETRG":
            if ((x['dep_prevu'][:1]=='D' or x['dep_prevu'][:1]=='F') and 'TABOT' in x['case15']):
                x['vol_a_transmettre'] = True
        else:
            x['vol_a_transmettre'] = False
        return x

    df_utile = df_utile.apply(TU_3_bis_element, axis=1)
    return df_utile

In [136]:
# def TU_3(df_utile):
#     def TU_3_element(x):
#         if x["depfinal"] == "ZZZZ":
#             x['PLN_valide'] = False
#             x['PLN_à_verifier_TU'] = False
#             x['invalidite_TU'].append("DEPAR2")
#             x['invalidite_TU'].append("TRANS1")
#         else:
#             if x["depfinal"][:2] in liste_origin_vol | x["depfinal"][:2] in liste_origin_vol:
#                 if trouver_code(x["depfinal"], liste_origin_vol) == "A":
#                     x['vol_a_transmettre'] = False
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "B":
#                     x['vol_a_transmettre'] = True
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "C":
#                     if x["AVR"] == 1:
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSC")
#                     else:
#                         x['vol_a_transmettre'] = False
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "D":
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSD")
#                 elif trouver_code(x["depfinal"], liste_origin_vol) == "E":
#                     if x["SVR"] == 1:
#                         x['vol_a_transmettre'] = False
#                     else:
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSE")
#                 else:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSI")
#             else:
#                 if x["AVR"] == 1:
#                     x['vol_a_transmettre'] = True
#                 else:
#                     if x["balisefinal"] == "PTGEO":
#                         x['vol_a_transmettre'] = "ADET"
#                         x["PLN_valide"] = False
#                         x['invalidite_TU'].append("TRANSP")
#                     else:
#                         x['vol_a_transmettre'] = False
#     df_utile = df_utile.apply(TU_3_element, axis=1)
#     return df_utile

In [137]:
# def TU_3(df_utile):
#     def TU_3_element(x):
#         dep_value = None
#         balise_value = None

#         # Vérification des colonnes de départ
#         for col in ["deprealise", "depfinal", "depprevu"]:
#             if col in x and not pd.isna(x[col]) and x[col] != "":
#                 dep_value = x[col]
#                 break
#         for col in ["baliserealise", "balisefinal", "baliseprevu"]:
#             if col in x and not pd.isna(x[col]) and x[col] != "":
#                 balise_value = x[col]
#                 break

#         if dep_value == "ZZZZ":
#             x['PLN_valide'] = False
#             x['PLN_à_verifier_TU'] = False
#             x['invalidite_TU'].extend(["DEPAR2", "TRANS1"])
#         elif dep_value:
#             dep_code = trouver_code(dep_value, liste_origin_vol)
#             if dep_code == "A":
#                 x['vol_a_transmettre'] = False
#             elif dep_code == "B":
#                 x['vol_a_transmettre'] = True
#             elif dep_code == "C":
#                 if x["AVR"] == 1:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSC")
#                 else:
#                     x['vol_a_transmettre'] = False
#             elif dep_code == "D":
#                 x['vol_a_transmettre'] = "ADET"
#                 x["PLN_valide"] = False
#                 x['invalidite_TU'].append("TRANSD")
#             elif dep_code == "E":
#                 if x["SVR"] == 1:
#                     x['vol_a_transmettre'] = False
#                 else:
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSE")
#             else:
#                 x['vol_a_transmettre'] = "ADET"
#                 x["PLN_valide"] = False
#                 x['invalidite_TU'].append("TRANSI")
#         else:
#             if x["AVR"] == 1:
#                 x['vol_a_transmettre'] = True
#             else:
#                 if balise_value == "PTGEO":
#                     x['vol_a_transmettre'] = "ADET"
#                     x["PLN_valide"] = False
#                     x['invalidite_TU'].append("TRANSP")
#                 else:
#                     x['vol_a_transmettre'] = False

#         return x

#     df_utile = df_utile.apply(TU_3_element, axis=1)
#     return df_utile

##  Algo 4

In [138]:
def TU_4(df_utile):
    def TU_4_element(x):
        x['vol_a_transmettre'] = True
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        for col in ["arr_realise", "arr_final", "arr_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                arr_value = x[col]
                break
        for col in ["plnActive_realise", "plnActive_final", "plnActive_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                plnActive = x[col]
                break

        if dep_value in AERODROMES_D_APPROCHE["Code terrain"].to_list():
            x["vol_approche"] = True

        if arr_value[:2] == "LF":
            x["vol_interieur"] = True

        elif  plnActive == 0 and dep_value in AERODROMES_FRONTALIERS["Code terrain"].to_list():
            x["vol_frontalier"] = True

        return x
    df_utile = df_utile.apply(TU_4_element, axis=1)
    return df_utile

In [139]:
# def TU_4(df_utile):
#     def TU_4_element(x):
#         x['vol_a_transmettre'] = True
#         if x["depfinal"] in AERODROMES_D_APPROCHE:
#             x["vol_approche"] = True
#         if x["arrfinal"][:2] == "LF":
#             x["vol_interieur"] == True
#         elif x["PLN_activefinal"] != 1 & x["depfinal"] in AERODROMES_FRONTALIERS:
#             x["vol_frontalier"] == True
#     df_utile = df_utile.apply(TU_4_element, axis=1)
#     return df_utile

## Algo 5

In [140]:
def TU_5(df_utile):
    def TU_5_element(x):
        # Vérification des colonnes balise et regleVol
        balise_value = None
        regleVol_value = None
        dep_value = None
        for col in ["baliserealise", "balisefinal", "baliseprevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                balise_value = x[col]
                break
        for col in ["regleVol_realise", "regleVol_final", "regleVol_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                regleVol_value = x[col]
                break
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in x and not pd.isna(x[col]) and x[col] != "":
                dep_value = x[col]
                break
        # Conditions basées sur les valeurs trouvées
        if x["typePln"] == "VFR" and balise_value.startswith('VFR') and regleVol_value == "V":
            x['vol_a_transmettre'] = False
        elif x["typePln"] in ["APL", "FPL", "FIH", "FII"] and balise_value.startswith('VFR'):
            x['vol_a_transmettre'] = False
            x['vol_VFR'] = True
            x["typePln"] = "VFR"
        else:
            if not x.get('vol_VFR', False) and regleVol_value == "V" and x.get('vol_a_transmettre', True):
                x["PLN_valide"] = False
                x['invalidite_TU'].append("TRANS0")
            elif dep_value and dep_value[:2] != "LF" and not x.get('vol_a_transmettre', True) and regleVol_value == "Z":
                x["PLN_valide"] = False
                x['invalidite_TU'].append("TRANS2")
                x["typePln"] = "AFI"
            if not x.get('vol_VFR', False) and any(s.startswith('TRANS') for s in x['invalidite_TU']):
                x['vol_a_transmettre'] = "ADET"
        if x["typePln"] in ["APL", "FPL"] and regleVol_value in ["Y", "Z"] and x.get('vol_a_transmettre', True):
            x["code_d_exoneration"] = "Y"
        return x
    df_utile = df_utile.apply(TU_5_element, axis=1)
    return df_utile


In [141]:
# def TU_5(df_utile):
#     def TU_5_element(x):
#         if x["typePln"] == "VFR" and x["baliseprevu"].str.startswith('VFR') and x["regleVol_prevu"] == "V":
#             x['vol_a_transmettre'] = False
#         elif (x["typePln"] == "APL" or x["typePln"] == "FPL" or x["typePln"] == "FIH" or x["typePln"] == "FII") and x["baliseprevu"].str.startswith('VFR'):
#             x['vol_a_transmettre'] = False
#             x['vol_VFR'] = True
#             x["typePln"] = "VFR"
#         else:
#             if x['vol_VFR'] == False and x["regleVol_prevu"] == "V" and x['vol_a_transmettre'] == True:
#                 x["PLN_valide"] == False
#                 x['invalidite_TU'].append("TRANS0")
#             elif x["depprevu"][:2] != "LF" and x['vol_a_transmettre'] == False and x["regleVol_prevu"] == "Z":
#                 x["PLN_valide"] == False
#                 x['invalidite_TU'].append("TRANS2")
#                 x["typePln"] = "AFI"
#             if x['vol_VFR'] == False and any(s.startswith('TRANS') for s in x['invalidite_TU']):
#                 x['vol_a_transmettre'] = "ADET"
#         if (x["typePln"] == "APL" or x["typePln"] == "FPL") and ( x["regleVol_prevu"] == "Y" or  x["regleVol_prevu"] == "Z") and x['vol_a_transmettre'] == True:
#             x["code_d_exoneration"] = "Y"
#     df_utile = df_utile.apply(TU_5_element, axis=1)
#     return df_utile

## Algo 6

In [142]:
def TU_65_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    type_avion_value = get_valid_value(x, ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"])
    if type_avion_value in AERONEFS_STRICTEMENT_MILITAIRES['Type avion'].tolist():
        x['type_d_avion_militaire'] = True
        x['PLN_valide'] = False
        x['invalidite_TU'].extend(["TYPA19", "INDI19"])
        filtre_avion = AERONEFS_STRICTEMENT_MILITAIRES[AERONEFS_STRICTEMENT_MILITAIRES["Type avion"] == type_avion_value]
        if not pd.isna(filtre_avion["Code exoneration"].iloc[0]):
            x["code_d_exoneration"] = filtre_avion["Code exoneration"].iloc[0]
            x['code_exploitant'] = filtre_avion["Code exploitant"].iloc[0]
        else:
            x['code_exploitant'] = 'Z'
    return x

In [143]:
def TU_64_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    type_avion_value = get_valid_value(x, ["typeAvion_realise", "typeAvion_final", "typeAvion_prevu"])
    OPR = None
    if not(pd.isna(x["case18"])):
        OPR = trouver_case18("OPR/", x["case18"])
    if not(pd.isna(x["case18"])) and OPR != "NULL" and OPR in OPERATEURS_MILITAIRES['Nom operateur'].tolist():
        operateur_militaire = OPERATEURS_MILITAIRES[OPERATEURS_MILITAIRES["Nom operateur"] == OPR]
        x["code_d_exoneration"] = operateur_militaire["Code exoneration"].iloc[0]
        x['code_exploitant'] = operateur_militaire["Code exploitant"].iloc[0]
    else:
        if type_avion_value in AERONEFS_STRICTEMENT_MILITAIRES['Type avion'].tolist():
            x['type_d_avion_militaire'] = True
            avion_militaire = AERONEFS_STRICTEMENT_MILITAIRES[AERONEFS_STRICTEMENT_MILITAIRES["Type avion"] == type_avion_value]
            if not pd.isna(avion_militaire["Code exoneration"].iloc[0]):
                x["code_d_exoneration"] = avion_militaire["Code exoneration"].iloc[0]
                x['code_exploitant'] = avion_militaire["Code exploitant"].iloc[0]
            else:
                x['code_exploitant'] = 'Z'
        x['PLN_valide'] = False
        x['invalidite_TU'].extend(["EXO19", "INDI19", "OPR19"])
    return x

In [144]:
def TU_63_element(x):
    # Fonction pour trouver une valeur valide dans les colonnes multiples
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and trouver_pattern(call_sign_value, IMMATRICULATION_CORRECTE["Immatriculation"]):
        if trouver_pattern(call_sign_value, IMMATRICULATION_A_VERIFIER["Immatriculation"]):
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["IVDIC$", "EXO$", "OPR$"])
        else:
            x['type_d_indicatif'] = "IM"
        x = TU_65_element(x)
    else:
        x = TU_64_element(x)
    return x

In [145]:
def TU_62_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and call_sign_value[:2] in COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"].values:
        x['code_exploitant'] = COMPAGNIES_BIGRAMMES_ET_SUFFIXES[COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"] == call_sign_value[:2]]["Code exploitant"].iloc[0]
        if x['vol_interieur'] == True:
            x["code_d_exoneration"] = COMPAGNIES_BIGRAMMES_ET_SUFFIXES[COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Indicatif"] == call_sign_value[:2]]["Code exoneration"].iloc[0]
            x['compagnie_française'] = True
            x['type_d_indicatif'] = "BI"
        else:
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["INDIC%", "EXO%", "OPR%"])
        x = TU_65_element(x)
    return x

In [146]:
def TU_612_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None

    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])

    if call_sign_value and re.search(r'\d[A-Z]$', call_sign_value) and call_sign_value[-1] == -1:
        x['PLN_valide'] = False
        x['invalidite_TU'].append("INDIC6")
    x = TU_65_element(x)
    return x

In [147]:
def TU_611_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value and call_sign_value[-1] in LETTRES_AIR_FRANCE['Lettre'].tolist():
        code_air_france = LETTRES_AIR_FRANCE[LETTRES_AIR_FRANCE['Lettre'] == call_sign_value[-1]]["Code exo/trait"].iloc[0]
        if code_air_france == 2:
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["INDIC8", "EXO8"])
        elif not code_air_france.isdigit():
            x["code_d_exoneration"] = code_air_france
    else:
        x['PLN_valide'] = False
        x['invalidite_TU'].append("EXO10")
    x = TU_65_element(x)
    return x

In [148]:
def TU_61_element(x):
    def get_valid_value(element, columns):
        for col in columns:
            if col in element and not pd.isna(element[col]) and element[col] != "":
                return element[col]
        return None
    call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
    if call_sign_value[:3] in COMPAGNIES_AVEC_TRIGRAMME["Trigramme"].tolist():
        x['type_d_indicatif'] = "TR"
        if trouver_pattern(call_sign_value, INDICATIFS_TRIGRAMME_A_VERIFIER["Indicatif"]):
            x['PLN_valide'] = False
            x['invalidite_TU'].extend(["IVDIC5", "EXO5"])
        elif call_sign_value[:3] == "AFR":
            x['compagnie_française'] = True
            if re.search(r'\d[A-Z]$', call_sign_value):
                x = TU_611_element(x)
            else:
                x = TU_65_element(x)
        else:
            x = TU_612_element(x)
    else:
        x = TU_64_element(x)
    return x

In [149]:
def TU_6(df_utile):
    def TU_6_element(x):
        def get_valid_value(element, columns):
            for col in columns:
                if col in element and not pd.isna(element[col]) and element[col] != "":
                    return element[col]
            return None
        call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
        if call_sign_value and trouver_pattern(call_sign_value, INDICATIF_A_CODE_AUTO["Indicatif"].tolist()):
            Indicatif = renvoie_pattern(call_sign_value, INDICATIF_A_CODE_AUTO["Indicatif"].tolist())
            x["code_d_exoneration"] = INDICATIF_A_CODE_AUTO[INDICATIF_A_CODE_AUTO["Indicatif"] == Indicatif]["Code exoneration"].iloc[0]
            x['code_exploitant'] = INDICATIF_A_CODE_AUTO[INDICATIF_A_CODE_AUTO["Indicatif"] == Indicatif]["Code exploitant"].iloc[0]
            if x["code_d_exoneration"] in ["X", "M"]:
                x['type_d_avion_militaire'] = True
        else:
            if trouver_pattern(call_sign_value, INDICATIFS_A_STRUCTURE_TRIGRAMME_CORRECTE["Indicatif"].tolist()):
                x = TU_61_element(x)
            elif re.match(r'^[A-Z]{2}\d{3}[A-Z]{2}$', call_sign_value) and call_sign_value[:2] in COMPAGNIES_BIGRAMMES_ET_SUFFIXES["Bigramme"].tolist():
                x = TU_62_element(x)
            else:
                x = TU_63_element(x)
        return x
    df_utile = df_utile.apply(TU_6_element, axis=1)
    return df_utile

## Algo 7

In [150]:
# Chaîne d'entrée
def trouver_case18(prefix, input_string):
    index_find = input_string.find(prefix)
    if index_find == -1:
        return "NULL"
    else:
        start_index = index_find + len(prefix)
        end_index = input_string.find(' ', start_index)
        opr_code = input_string[start_index:end_index]
        return opr_code

In [151]:
# def TU_7(df_utile):
#     def TU_7_element(x):
#         if x["PLN_activeprevu"] == "0" and x['vol_a_transmettre'] == True:
#             x['PLN_valide'] = False
#         if x["callSign_prevu"] == "APL" or x["callSign_prevu"] == "FPL":
#             if x['vol_a_transmettre'] == True:
#                 if x["code_d_exoneration"] == "Y" or ["code_d_exoneration"] == "Z":
#                     if x['typeVol_prevu'] == "M":
#                         x['PLN_valide'] = False
#                         x['invalidite_TU'].append("EXO10")
#                         x['invalidite_TU'].append("OPR10")
#                     else:
#                         if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x['vol_interieur'] == True:
#                             x["code_d_exoneration"] = "T"
#                         elif x['typeVol_prevu'] == "X" and trouver_case18("RMK/", x["case18"]) != "EVASAN":
#                             x['PLN_valide'] = trouver_case18
#                             x['invalidite_TU'].append("EXO12")
#             else:
#                 if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x['vol_interieur'] == True and (x["code_d_exoneration"] == "Y" or ["code_d_exoneration"] == "Z"):
#                     x["code_d_exoneration"] = "T"

#     df_utile = df_utile.apply(TU_7_element, axis=1)
#     return df_utile

In [152]:
def TU_7(df_utile):
    def TU_7_element(x):
        # Fonction pour trouver une valeur valide dans les colonnes multiples
        def get_valid_value(element, columns):
            for col in columns:
                if col in element and not(element[[col]].isna().iloc[0]):
                    return element[col]
            return None

        PLN_active_value = get_valid_value(x, ["plnActive_realise", "plnActive_final", "plnActive_prevu"])
        call_sign_value = get_valid_value(x, ["callSign_realise", "callSign_final", "callSign_prevu"])
        type_vol_value = get_valid_value(x, ["typeVol_realise", "typeVol_final", "typeVol_prevu"])

        if PLN_active_value == "0" and x.get('vol_a_transmettre', False):
            x['PLN_valide'] = False

        if call_sign_value in ["APL", "FPL"]:
            if x.get('vol_a_transmettre', False):
                if code_exoneration_value in ["Y", "Z"]:
                    if type_vol_value == "M":
                        x['PLN_valide'] = False
                        x['invalidite_TU'].extend(["EXO10", "OPR10"])
                    else:
                        if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x.get('vol_interieur', False):
                            x["code_d_exoneration"] = "T"
                        elif type_vol_value == "X" and trouver_case18("RMK/", x["case18"]) != "EVASAN":
                            x['PLN_valide'] = False
                            x['invalidite_TU'].append("EXO12")
            else:
                if trouver_case18("RMK/", x["case18"]) == "TRAINING" and x.get('vol_interieur', False) and code_exoneration_value in ["Y", "Z"]:
                    x["code_d_exoneration"] = "T"
        return x
    df_utile = df_utile.apply(TU_7_element, axis=1)
    return df_utile

## Traitement unitaire complet

In [153]:
def traitement_unitaire(df_utile):
    df_utile = TU_init(df_utile)
    df_utile = TU_1(df_utile)
    df_utile = TU_2(df_utile)

    def check_depfinal_prefix(row):
        for col in ["dep_realise", "dep_final", "dep_prevu"]:
            if col in row and not pd.isna(row[col]) and row[col] != "":
                if row[col][:2] == "LF":
                    return True
        return False

    def apply_transformations(row):
        if check_depfinal_prefix(row):
            return TU_4(pd.DataFrame([row])).iloc[0]
        else:
            return TU_3_bis(pd.DataFrame([row])).iloc[0]

    # Appliquer les transformations par ligne
    df_utile = df_utile.apply(apply_transformations, axis=1)

    df_utile = TU_5(df_utile)
    df_utile = TU_6(df_utile)
    df_utile = TU_7(df_utile)
    return df_utile

# Transformation des données RDVC

In [154]:
jour_1_utile_inutile = traitement_utile_inutile(jour_1)
jour_2_utile_inutile = traitement_utile_inutile(jour_2)
jour_3_utile_inutile = traitement_utile_inutile(jour_3)

In [155]:
jour_1_utile = jour_1_utile_inutile[jour_1_utile_inutile["utile_inutile"] == "UTI"].copy()
jour_2_utile = jour_2_utile_inutile[jour_2_utile_inutile["utile_inutile"] == "UTI"].copy()
jour_3_utile = jour_3_utile_inutile[jour_3_utile_inutile["utile_inutile"] == "UTI"].copy()

In [156]:
jour_1_utile_traitee = traitement_unitaire(jour_1_utile)
jour_2_utile_traitee = traitement_unitaire(jour_2_utile)
jour_3_utile_traitee = traitement_unitaire(jour_3_utile)

Fonction
ajouter pour chaque df une colonne doublon à False
Verifier que si un call sign et une heure de reference à 19mn pret (à parametrer) indiquer doublon sur l'un des vols seulement

In [157]:
seuil_doublon = 19

In [158]:
import pandas as pd

def ajouter_colonne_doublon(df, tolerance_minutes=19):
    df['doublon'] = False

    df = df.sort_values(by=['callSign_prevu', 'heure_de_reference']).reset_index(drop=True)
    i = 1
    while i < len(df):
        if df.at[i, 'callSign_prevu'] == df.at[i - 1, 'callSign_prevu']:
            delta = abs((df.at[i, 'heure_de_reference'] - df.at[i - 1, 'heure_de_reference']))
            if delta <= tolerance_minutes:
                df = df.drop(i).reset_index(drop=True)
                continue  # Réévaluer le même index i après la suppression et la réinitialisation de l'index
        i += 1
    return df

def supprimer_doublons(df):
    return df[df['doublon'] == False].drop(columns=['doublon'])

In [159]:
jour_1_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_1_utile_traitee)
jour_2_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_2_utile_traitee)
jour_3_utile_traitee_avec_doublons_false = ajouter_colonne_doublon(jour_3_utile_traitee)

jour_1_utile_traitee_sans_doublons = supprimer_doublons(jour_1_utile_traitee_avec_doublons_false)
jour_2_utile_traitee_sans_doublons = supprimer_doublons(jour_2_utile_traitee_avec_doublons_false)
jour_3_utile_traitee_sans_doublons = supprimer_doublons(jour_3_utile_traitee_avec_doublons_false)

C:\Users\latti\AppData\Local\Temp\ipykernel_38372\4285868749.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False
C:\Users\latti\AppData\Local\Temp\ipykernel_38372\4285868749.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['doublon'] = False
C:\Users\latti\AppData\Local\Temp\ipykernel_38372\4285868749.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conc

In [160]:
def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return ""

def format_flight_message(row):
    sequence_number = f"{row.name + 1:04}"
    intro_correction_code = "F"
    valeur = int(get_valid_value(row, ['heure_de_reference']))
    heure = (valeur%1440)//60
    min = (valeur%1440)%60
    time_of_departure = f"{(100*heure + min):04}"
    aerodrome_of_departure = get_valid_value(row, ['dep_realise', 'dep_final', 'de_prevu']).ljust(4)
    aerodrome_of_arrival = get_valid_value(row, ['arr_realise', 'arr_final', 'arr_prevu']).ljust(4)
    flight_identification = get_valid_value(row, ['callSign_realise', 'callSign_final', 'callSign_prevu']).ljust(9)
    main_exemption_code = row["code_d_exoneration"].ljust(1)
    aircraft_type = get_valid_value(row, ['typeAvion_realise', 'typeAvion_final', 'typeAvion_prevu']).ljust(7)
    operator = str(row["code_exploitant"]).ljust(3)
    aircraft_registration = str(get_valid_value(row, ['work1realise', 'work1final', 'work1prevu'])).ljust(9)
    comment = ""
    est_off_block_date = get_valid_value(row, ['date_de_reference']).ljust(6)
    ifplid = get_valid_value(row, ['IFPL_realise', 'IFPL_final', 'IFPL_prevu']).ljust(9)
    initial_aerodrome_destination = ""
    charging_zone_overflown = ""
    entry_point_coordinates = ""
    exit_point_coordinates = ""
    supplementary_exemption_codes = ""
    source_icao_address = ""
    icao_address = get_valid_value(row, ['adresseModeS_realise', 'adresseModeS_final', 'adresseModeS_prevu']).ljust(6)
    additional_comment = ""
    front_alg_fr = row["front_alg_fr"]
    premier_plot_fr = str(row["premier_plot_fr"])
    modes_fr = str(row["modes_fr"])

    return {
        "Sequence number": sequence_number,
        "Code": intro_correction_code,
        "Time of departure/entry": time_of_departure,
        "Departure aerodrome": aerodrome_of_departure,
        "Arrival aerodrome": aerodrome_of_arrival,
        "Flight identification": flight_identification,
        "Main Exemption code": main_exemption_code,
        "Type of aircraft": aircraft_type,
        "Operator": operator,
        "Aircraft Registration": aircraft_registration,
        "Comment1": comment,
        "Flight date": est_off_block_date,
        "IFPLID": ifplid,
        "Planned_aerodrome": initial_aerodrome_destination,
        "Charging_zone_overflow": charging_zone_overflown,
        "Entry_point": entry_point_coordinates,
        "Exit_point": exit_point_coordinates,
        "Sup_exemption_code": supplementary_exemption_codes,
        "Source of the Aircraft Address": source_icao_address,
        "24-bit Aircraft Address": icao_address,
        "Comment2": additional_comment,
        "case7" : str(row["case7"]),
        "case8" : str(row["case8"]),
        "case9" : str(row["case9"]),
        "case10" : str(row["case10"]),
        "case13" : str(row["case13"]),
        "case15" : str(row["case15"]),
        "case16" : str(row["case16"]),
        "case18" : str(row["case18"]),
        "ccrArrival" : str(row["ccrArrival"]),
        "front_alg_fr" : front_alg_fr,
        "premier_plot_fr" : premier_plot_fr,
        "modes_fr" : modes_fr
    }


In [161]:
jour_123 = pd.concat([jour_1_utile_traitee_avec_doublons_false, jour_2_utile_traitee_avec_doublons_false, jour_3_utile_traitee_avec_doublons_false], ignore_index=True)
jour_123['date_de_reference'] = jour_123['date_de_reference'].dt.strftime('%y%m%d')
jour_123 = jour_123[jour_123["vol_a_transmettre"] == True]
target_date_str =dateAnalyse
target_date = datetime.strptime(target_date_str, "%d-%m-%Y")
target_date_str = target_date.strftime("%y%m%d")
jour_cible = jour_123[(jour_123["date_de_reference"] == target_date_str) | (jour_123["date_de_reference"] == "      ")]


In [162]:
data = jour_cible.copy()

In [163]:
jour_cible = data.copy()

In [164]:
nom_fichier_survol = f'../data/fusion_radar/data_survols_{datetime.strptime(dateAnalyse, "%d-%m-%Y").strftime("%Y%m%d")}.csv'
print(nom_fichier_survol)
data_survols = pd.read_csv(nom_fichier_survol, sep=',')
def conversion_dec_hexa(number):
    if number != number:  # Vérification pour NaN
        return 'NaN'
    else:
        hex_value = hex(int(number))[2:].upper()
        return hex_value.zfill(6)
data_survols['MODES_ADRESS'] = data_survols['MODES_ADRESS'].apply(conversion_dec_hexa)
# Créer les colonnes front_alg_fr, premier_plot_fr, modes_fr avec des NaN
jour_cible['front_alg_fr'] = np.nan
jour_cible['premier_plot_fr'] = np.nan
jour_cible['modes_fr'] = np.nan

# Parcourir chaque ligne de jour_cible
for i, row in jour_cible.iterrows():
    # Filtrer data_survols pour trouver les lignes où les critères sont respectés
    matched_rows = data_survols[
        (row['callSign_prevu'] == data_survols['FLPL_CALL_SIGN']) &
        (abs((100*(row['heure_de_reference']%1440)//60 + (row['heure_de_reference']%1440)%60) - data_survols['premier_plot']) < 500)
        # (row['arr_prevu'] == data_survols['FLPL_ARRV_AIRP'])
    ]

    if not matched_rows.empty:
        # Si une correspondance est trouvée, prendre la première (ou vous pouvez choisir une autre règle)
        match = matched_rows.iloc[0]

        # Mettre à jour les colonnes de jour_123
        jour_cible.at[i, 'front_alg_fr'] = match['frontiere_algerienne']
        jour_cible.at[i, 'premier_plot_fr'] = match['premier_plot']
        jour_cible.at[i, 'modes_fr'] = match['MODES_ADRESS']


../data/fusion_radar/data_survols_20240321.csv


C:\Users\latti\AppData\Local\Temp\ipykernel_38372\554829931.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  jour_cible.at[i, 'front_alg_fr'] = match['frontiere_algerienne']
C:\Users\latti\AppData\Local\Temp\ipykernel_38372\554829931.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4D4038' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  jour_cible.at[i, 'modes_fr'] = match['MODES_ADRESS']


In [165]:
jour_cible = jour_cible.apply(format_flight_message, axis=1).tolist()
jour_cible = pd.DataFrame(jour_cible)
jour_cible['Time of departure/entry'] = pd.to_numeric(jour_cible['Time of departure/entry'], errors='coerce')

In [166]:
modeS1 = pd.read_csv(os.path.join(chemin_dossier_table_systeme,"modeS1"), sep=';', skiprows=3)
modeS1 = modeS1[['Mode S', 'Type Avion']]
modeS1.rename(columns={
    'Type Avion': 'Type Avion Survol'
}, inplace=True)


jour_cible['Type_Avion_Survol'] = np.nan
# Parcourir chaque ligne de jour_cible
for i, row in jour_cible.iterrows():
    # Filtrer data_survols pour trouver les lignes où les critères sont respectés
    matched_rows = modeS1[
        (row['modes_fr'] == modeS1['Mode S'])
    ]

    if not matched_rows.empty:
        # Si une correspondance est trouvée, prendre la première (ou vous pouvez choisir une autre règle)
        match = matched_rows.iloc[0]
        # Mettre à jour les colonnes de jour_cible
        jour_cible.at[i, 'Type_Avion_Survol'] = match['Type Avion Survol']


C:\Users\latti\AppData\Local\Temp\ipykernel_38372\4033413320.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'C525' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  jour_cible.at[i, 'Type_Avion_Survol'] = match['Type Avion Survol']


In [167]:
# Sauvegarder le DataFrame en fichier CSV
# jour_cible.to_csv('jour_cible_FR.csv', index=False)

# # Télécharger le fichier CSV sur votre ordinateur local
# files.download('jour_cible_FR.csv')


In [168]:
jour_cible.shape[0]

2535

In [169]:
jour_cible[(jour_cible['24-bit Aircraft Address'] == "      ")].shape[0]

133

In [170]:
jour_cible[(jour_cible['modes_fr'] == "nan") & (jour_cible['24-bit Aircraft Address'] == "      ")].shape[0]

24

In [171]:
jour_cible.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Sequence number                 2535 non-null   object
 1   Code                            2535 non-null   object
 2   Time of departure/entry         2535 non-null   int64 
 3   Departure aerodrome             2535 non-null   object
 4   Arrival aerodrome               2535 non-null   object
 5   Flight identification           2535 non-null   object
 6   Main Exemption code             2535 non-null   object
 7   Type of aircraft                2535 non-null   object
 8   Operator                        2535 non-null   object
 9   Aircraft Registration           2535 non-null   object
 10  Comment1                        2535 non-null   object
 11  Flight date                     2535 non-null   object
 12  IFPLID                          2535 non-null   

In [172]:
# Supprimer les espaces en début et en fin de chaîne
jour_cible['Type of aircraft'] = jour_cible['Type of aircraft'].str.strip()
jour_cible['Type_Avion_Survol'] = jour_cible['Type_Avion_Survol'].str.strip()

# Filtrer les lignes où les valeurs des colonnes sont différentes
diff_rows = jour_cible[jour_cible['Type of aircraft'] != jour_cible['Type_Avion_Survol']]

# Afficher les lignes filtrées
print(diff_rows.iloc[0])

Sequence number                        7322
Code                                      F
Time of departure/entry                1756
Departure aerodrome                    LFML
Arrival aerodrome                      LFML
Flight identification             175Y     
Main Exemption code                       Z
Type of aircraft                       EC25
Operator                                Z  
Aircraft Registration                      
Comment1                                   
Flight date                          240321
IFPLID                                     
Planned_aerodrome                          
Charging_zone_overflow                     
Entry_point                                
Exit_point                                 
Sup_exemption_code                         
Source of the Aircraft Address             
24-bit Aircraft Address                    
Comment2                                   
case7                                   nan
case8                           

In [173]:
jour_cible.head()

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2,case7,case8,case9,case10,case13,case15,case16,case18,ccrArrival,front_alg_fr,premier_plot_fr,modes_fr,Type_Avion_Survol
0,7322,F,1756,LFML,LFML,175Y,Z,EC25,Z,,,240321,,,,,,,,,,nan,nan,nan,nan,nan,nan,nan,nan,MARS,NaN,nan,nan,NaN
1,7323,F,1058,LFPB,LFMD,3AMSR,Z,C525,Z,3AMSR,,240321,AA57016889,,,,,,,4D4038,,3AMSR,IG,C525/L,SBDFGRWYZ/HB1V1,LFPB1100,N0378F380 LATRA DCT LAMUT DCT UTUVA DCT LERGA/...,LFMD0109 LFMN,DAT/CPDLCX DOF/240321 EET/LFMM0028 IFP/MODESAS...,CDG,False,1058.0,4D4038,C525
2,7329,F,1636,LFLB,EBLG,AAB75P,Z,GL5T,Z,OOWLP,,240321,AA57104754,,,,,,,44DD90,,AAB75P,IN,GL5T/M,SBDE1E2E3FGHIJ1J3J4J5P2RWXY/LB2D1G1,LFLB1630,N0473F280 VENAT T330 GILIR/N0494F320 DCT PENDU...,EBLG0102 EHBK EBBR,CODE/44DD90 DOF/240321 EET/LFFF0005 LSAS0013 L...,LYON,False,1535.0,44DD90,GL5T
3,7333,F,1650,LFMQ,EBKT,AAB89X,Z,C25C,Z,OOLXA,,240321,AA57074540,,,,,,,44B301,,AAB89X,IN,C25C/M,SBDGRWXYZ/LB1,LFMQ1550,N0443F340 MTL UM976 MOPEM DCT CACHI DCT AVLON ...,EBKT0124 LFQQ EBBR,DAT/CPDLCX DOF/240321 EET/LFFF0005 EBBU0115 IF...,MARS,False,1547.0,44B301,C25C
4,7340,F,1049,LFPG,KJFK,AAL45,Z,B772,Z,N756A,,240321,AA57092794,,,,,,,AA305F,,AAL45,IS,B772/H,SDE1E3FGHIJ3J5M1P2RWXYZ/LB1D1,LFPG1025,N0480F340 LGL UN491 UPALO/N0477F350 UN585 FEJA...,KJFK0805 KEWR,CODE/AA305F DAT/CPDLCX 1FANS2PDC DOF/240321 EE...,CDG,False,1048.0,AA305F,B772


In [174]:
diff_rows.head()

,Sequence number,Code,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Main Exemption code,Type of aircraft,Operator,Aircraft Registration,Comment1,Flight date,IFPLID,Planned_aerodrome,Charging_zone_overflow,Entry_point,Exit_point,Sup_exemption_code,Source of the Aircraft Address,24-bit Aircraft Address,Comment2,case7,case8,case9,case10,case13,case15,case16,case18,ccrArrival,front_alg_fr,premier_plot_fr,modes_fr,Type_Avion_Survol
0,7322,F,1756,LFML,LFML,175Y,Z,EC25,Z,,,240321,,,,,,,,,,nan,nan,nan,nan,nan,nan,nan,nan,MARS,NaN,nan,nan,NaN
15,7374,F,1315,LFBD,EDDP,ABR760,Z,A306,Z,EILDC,,240321,AA57101304,,,,,,,4CAE23,,ABR760,IX,A306/H,SDFGHIJ3RVWXYZ/LB1,LFBD1300,N0442F330 CNA DCT ADABI DCT BOKNO DCT DIBES DC...,EDDP0206 EDDK,DAT/CPDLCX DOF/240321 EET/EBUR0109 EDUU0114 IF...,MERI,False,1321.0,4CAE23,NaN
541,8236,F,1526,LFBO,LFMM,AIB01BU,Z,A20N,Z,,,240321,,,,,,,,,,AIB01BU,IX,A20N/M,LOVY/C,LFBO1545,N0250F070 FINX,LFMM0030 LFBT,STS/FLTCK DOF/240321 RMK/RETOUCHE CHECK FLIGHT...,TOUL,False,1526.0,380F7A,NaN
542,8237,F,1615,LFBB,LFBO,AIB01BU,Z,A20N,Z,,,240321,,,,,,,,,,AIB01BU,IX,A20N/M,LOVY/C,LFBB1615,N0250F080 XTLW SU,LFBO0010 LFBT,STS/FLTCK DOF/240321 RMK/RETOUCHE CHECK FLIGHT...,TOUL,False,1526.0,380F7A,NaN
543,8238,F,1440,LFBO,LFBB,AIB02DJ,Z,A20N,Z,,,240321,,,,,,,,,,AIB02DJ,IX,A20N/M,LOVY/C,LFBO1445,N0250F070 LACX,LFBB0100 LFBT,STS/FLTCK DOF/240321)\n,TOUL,False,1447.0,38111A,NaN


In [175]:
diff_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140 entries, 0 to 2533
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Sequence number                 140 non-null    object
 1   Code                            140 non-null    object
 2   Time of departure/entry         140 non-null    int64 
 3   Departure aerodrome             140 non-null    object
 4   Arrival aerodrome               140 non-null    object
 5   Flight identification           140 non-null    object
 6   Main Exemption code             140 non-null    object
 7   Type of aircraft                140 non-null    object
 8   Operator                        140 non-null    object
 9   Aircraft Registration           140 non-null    object
 10  Comment1                        140 non-null    object
 11  Flight date                     140 non-null    object
 12  IFPLID                          140 non-null    object

In [176]:
# Forcer la conversion des valeurs 'NaN' en np.nan
diff_rows['Type_Avion_Survol'] = diff_rows['Type_Avion_Survol'].apply(lambda x: np.nan if pd.isna(x) or x == 'nan' else x)

# Compter le nombre de valeurs NaN dans la colonne 'Type_Avion_Survol'
nan_count = diff_rows['Type_Avion_Survol'].isnull().sum()

# Afficher le résultat
print(f"Nombre de valeurs NaN dans la colonne 'Type_Avion_Survol' : {nan_count}")

Nombre de valeurs NaN dans la colonne 'Type_Avion_Survol' : 114


C:\Users\latti\AppData\Local\Temp\ipykernel_38372\579926527.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_rows['Type_Avion_Survol'] = diff_rows['Type_Avion_Survol'].apply(lambda x: np.nan if pd.isna(x) or x == 'nan' else x)


In [177]:
# Supprimer les lignes où les valeurs de la colonne 'Type_Avion_Survol' sont NaN
diff_rows = diff_rows.dropna(subset=['Type_Avion_Survol'])



In [178]:
# Conserver uniquement les colonnes spécifiées dans diff_rows
colonnes_a_conserver = ['Time of departure/entry', 'Departure aerodrome', 'Arrival aerodrome', 'Flight identification', 'Type of aircraft', 'Aircraft Registration', '24-bit Aircraft Address', 'modes_fr', 'Type_Avion_Survol']
diff_rows = diff_rows[colonnes_a_conserver]

# Afficher le DataFrame mis à jour
diff_rows.head(40)

,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Type of aircraft,Aircraft Registration,24-bit Aircraft Address,modes_fr,Type_Avion_Survol
607,1603,LFBO,EGLL,BAW379,A320,GTTNH,4075FD,4075FD,A20N
991,1220,LFPN,LFOB,EAP65,P68,FHUBG,39D026,3990C8,C77R
992,1346,LFOB,LFPN,EAP65,P68,FHUBG,39D026,3990C8,C77R
1121,1521,LFRS,GMMX,EJU65KD,A320,OELKC,440484,440484,A319
1183,954,LFBO,LFRS,EJU897R,A320,OELKM,440625,440625,A319
1306,942,LFBF,LFBT,FBLEB,PA23,FBLEB,382C81,382C81,PA27
1361,1455,LFPT,LFPN,FGTFI,TB20,,394CA8,394CA8,TOBA
1362,1606,LFPN,LFPT,FGTFI,TB20,,,394CA8,TOBA
1365,800,LFMA,LFMV,FGVCL,P32R,,395562,395562,B190
1366,900,LFMV,LFMA,FGVCL,P32R,,,395562,B190


In [179]:
diff_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 607 to 2226
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Time of departure/entry  26 non-null     int64 
 1   Departure aerodrome      26 non-null     object
 2   Arrival aerodrome        26 non-null     object
 3   Flight identification    26 non-null     object
 4   Type of aircraft         26 non-null     object
 5   Aircraft Registration    26 non-null     object
 6   24-bit Aircraft Address  26 non-null     object
 7   modes_fr                 26 non-null     object
 8   Type_Avion_Survol        26 non-null     object
dtypes: int64(1), object(8)
memory usage: 2.0+ KB


In [180]:
# Ajouter une nouvelle colonne pour indiquer les conditions spécifiées
def check_aircraft_type(row):
    type_of_aircraft = row['Type of aircraft']
    type_avion_survol = row['Type_Avion_Survol']
    
    in_type_of_aircraft = type_of_aircraft in AERONEFS_DE_MOINS_DE_2_TONNES['Type avion'].values
    in_type_avion_survol = type_avion_survol in AERONEFS_DE_MOINS_DE_2_TONNES['Type avion'].values
    
    if in_type_of_aircraft and in_type_avion_survol:
        return 'type avion moins de 2T'
    elif in_type_of_aircraft:
        return 'stan moins de 2 tonnes seulement'
    elif in_type_avion_survol:
        return 'FR moins de 2 tonnes seulement'
    else:
        return 'plus de 2 T'

diff_rows['Type_avion_status'] = diff_rows.apply(check_aircraft_type, axis=1)

In [181]:
diff_rows.head(27)

,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Flight identification,Type of aircraft,Aircraft Registration,24-bit Aircraft Address,modes_fr,Type_Avion_Survol,Type_avion_status
607,1603,LFBO,EGLL,BAW379,A320,GTTNH,4075FD,4075FD,A20N,plus de 2 T
991,1220,LFPN,LFOB,EAP65,P68,FHUBG,39D026,3990C8,C77R,type avion moins de 2T
992,1346,LFOB,LFPN,EAP65,P68,FHUBG,39D026,3990C8,C77R,type avion moins de 2T
1121,1521,LFRS,GMMX,EJU65KD,A320,OELKC,440484,440484,A319,plus de 2 T
1183,954,LFBO,LFRS,EJU897R,A320,OELKM,440625,440625,A319,plus de 2 T
1306,942,LFBF,LFBT,FBLEB,PA23,FBLEB,382C81,382C81,PA27,stan moins de 2 tonnes seulement
1361,1455,LFPT,LFPN,FGTFI,TB20,,394CA8,394CA8,TOBA,type avion moins de 2T
1362,1606,LFPN,LFPT,FGTFI,TB20,,,394CA8,TOBA,type avion moins de 2T
1365,800,LFMA,LFMV,FGVCL,P32R,,395562,395562,B190,stan moins de 2 tonnes seulement
1366,900,LFMV,LFMA,FGVCL,P32R,,,395562,B190,stan moins de 2 tonnes seulement


# Abacus analyse mode S

## Fonction

In [182]:
def get_valid_value(element, columns):
        for col in columns:
            if col in element and not(element[[col]].isna().iloc[0]):
                return element[col]
        return ""

def format_flight_message2(row):
    sequence_number = f"{row.name + 1:04}"
    intro_correction_code = "F"
    time_of_departure = f"{(int(get_valid_value(row, ['Time of departure/entry']))):04}"
    aerodrome_of_departure = get_valid_value(row, ['Departure aerodrome']).ljust(4)
    aerodrome_of_arrival = get_valid_value(row, ['Arrival aerodrome']).ljust(4)
    flight_identification = get_valid_value(row, ['Flight identification']).ljust(9)
    main_exemption_code = row['Main Exemption Code'].ljust(1)
    aircraft_type = get_valid_value(row, ['Type of aircraft']).ljust(7)
    operator = row['Main Exemption Code'].ljust(3)
    aircraft_registration = get_valid_value(row, ['Aircraft Registration']).ljust(9)
    comment = ""
    est_off_block_date = str(get_valid_value(row, ['Flight date'])).ljust(6)
    ifplid = get_valid_value(row, ['IFPLID']).ljust(9)
    initial_aerodrome_destination = ""
    charging_zone_overflown = ""
    entry_point_coordinates = ""
    exit_point_coordinates = ""
    supplementary_exemption_codes = ""
    source_icao_address = row['Source of the Aircraft Address']
    icao_address = row['24-bit Aircraft Address']
    additional_comment = ""

    return {
        "Sequence number": sequence_number,
        "Code": intro_correction_code,
        "Time of departure/entry": time_of_departure,
        "Departure aerodrome": aerodrome_of_departure,
        "Arrival aerodrome": aerodrome_of_arrival,
        "Flight identification": flight_identification,
        "Main Exemption code": main_exemption_code,
        "Type of aircraft": aircraft_type,
        "Operator": operator,
        "Aircraft Registration": aircraft_registration,
        "Comment1": comment,
        "Flight date": est_off_block_date,
        "IFPLID": ifplid,
        "Planned_aerodrome": initial_aerodrome_destination,
        "Charging_zone_overflow": charging_zone_overflown,
        "Entry_point": entry_point_coordinates,
        "Exit_point": exit_point_coordinates,
        "Sup_exemption_code": supplementary_exemption_codes,
        "Source of the Aircraft Address": source_icao_address,
        "24-bit Aircraft Address": icao_address,
        "Comment2": additional_comment
    }

## Analyse

In [183]:
liste_noms_colonnes = ['Origin of message','Flight date','Sequence number','Time of departure/entry','Departure aerodrome','Arrival aerodrome','Type of aircraft','Flight identification','Aircraft Registration','User ICAO Code','User Number','VAT Number','User Nationality','Correction Code','IFPLID','24-bit Aircraft Address','Source of the Aircraft Address','Main Exemption Code','1st Supp. Ex. Code','2nd Supp. Ex. Code','3rd Supp. Ex. Code','Flight Message ID','Claim Number','Pro forma number','Pro forma line number','Bill or Credit Note Reference','MTOW','Weight Coefficient','Distance Coefficient','Service Units','National Route Charge in BZ','Admin. Route Charge in BZ','National Route Charge for SP','Admin. Route Charge for SP','Exemption Indicator','Exemption Code','Flight category','VAT Doc. Ref.','Original VAT Doc. Ref','VAT Rate','VAT Code','VAT on National Route Charge for SP','VAT on Admin. Route Charge for SP']
ABACUS_Mars = pd.read_csv(os.path.join(chemin_dossier_abacus,'LF20LF10%S%ABACUS_FLSPBZ03_2403.TXT'), sep=';', usecols = liste_noms_colonnes)
ABACUS_Mars.info()
ABACUS = ABACUS_Mars[(ABACUS_Mars["Flight date"] == int(target_date_str)) & (ABACUS_Mars["Origin of message"] == "LF")]


C:\Users\latti\AppData\Local\Temp\ipykernel_38372\1265566308.py:2: DtypeWarning: Columns (18,35) have mixed types. Specify dtype option on import or set low_memory=False.
  ABACUS_Mars = pd.read_csv(os.path.join(chemin_dossier_abacus,'LF20LF10%S%ABACUS_FLSPBZ03_2403.TXT'), sep=';', usecols = liste_noms_colonnes)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253940 entries, 0 to 253939
Data columns (total 43 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Origin of message                    253940 non-null  object 
 1   Flight date                          253940 non-null  int64  
 2   Sequence number                      253940 non-null  int64  
 3   Time of departure/entry              253940 non-null  int64  
 4   Departure aerodrome                  253940 non-null  object 
 5   Arrival aerodrome                    253940 non-null  object 
 6   Type of aircraft                     253940 non-null  object 
 7   Flight identification                253940 non-null  object 
 8   User ICAO Code                       243911 non-null  object 
 9   User Number                          253940 non-null  int64  
 10  VAT Number                           240860 non-null  object 
 11  User National

In [184]:
ABACUS_Mars

,Origin of message,Flight date,Sequence number,Time of departure/entry,Departure aerodrome,Arrival aerodrome,Type of aircraft,Flight identification,User ICAO Code,User Number,VAT Number,User Nationality,Correction Code,IFPLID,24-bit Aircraft Address,Source of the Aircraft Address,Main Exemption Code,1st Supp. Ex. Code,2nd Supp. Ex. Code,3rd Supp. Ex. Code,Flight Message ID,Pro forma number,Pro forma line number,Bill or Credit Note Reference,Aircraft Registration,MTOW,Weight Coefficient,Distance Coefficient,Service Units,National Route Charge in BZ,Admin. Route Charge in BZ,National Route Charge for SP,Admin. Route Charge for SP,Exemption Indicator,Exemption Code,Flight category,VAT Doc. Ref.,Original VAT Doc. Ref,VAT Rate,VAT Code,VAT on National Route Charge for SP,VAT on Admin. Route Charge for SP,Claim Number
0,LF,240301,836,541,LFBR,LFMV,M20P,DEIKE,NaN,22,NaN,EB,1.0,AA56536075,3D1AD0,1.0,Z,NaN,NaN,NaN,410384460,1,2,02/000022/22/2403/01,DEIKE,"0001,5","000,17","002,56","000,4352","00030,84","0000,05","00030,84","0000,05",1.0,2,1.0,NaN,NaN,"00,00",NaN,"00000,00","0000,00",NaN
1,LF,240301,1341,719,LFOT,LFLX,P06T,FHMZA,NaN,22,NaN,EB,1.0,AA56528428,39B320,1.0,Z,NaN,NaN,NaN,410384965,1,13,02/000022/22/2403/01,FHMZA,"0001,2","000,15","000,58","000,0870","00006,17","0000,01","00006,17","0000,01",1.0,2,1.0,NaN,NaN,"00,00",NaN,"00000,00","0000,00",NaN
2,LS,240301,74,730,LFLP,LSGL,SR22,OKCIR,NaN,22,NaN,EB,1.0,AA56493885,NaN,NaN,Z,NaN,NaN,NaN,410226168,1,17,02/000022/22/2403/01,OKCIR,"0001,6","000,18","000,21","000,0378","00002,68","0000,00","00002,68","0000,00",1.0,2,0.0,NaN,NaN,"00,00",NaN,"00000,00","0000,00",NaN
3,LF,240301,1328,730,LFMP,LFMP,P28R,FHFEA,NaN,22,NaN,EB,1.0,AA56532620,399480,1.0,Z,NaN,NaN,NaN,410384952,1,18,02/000022/22/2403/01,FHFEA,"0001,2","000,15","000,32","000,0480","00003,40","0000,01","00003,40","0000,01",1.0,2,2.0,NaN,NaN,"00,00",NaN,"00000,00","0000,00",NaN
4,LF,240301,1266,754,LFMD,LFMQ,P28A,FGJLR,NaN,22,NaN,EB,1.0,AA56532629,392571,1.0,Z,NaN,NaN,NaN,410384890,1,25,02/000022/22/2403/01,FGJLR,"0001,2","000,15","000,60","000,0900","00006,38","0000,01","00006,38","0000,01",1.0,2,1.0,NaN,NaN,"00,00",NaN,"00000,00","0000,00",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253935,LS,231212,365,1448,LSHI,LFSB,EC45,RGA02,SAZ,2638,CHE-116.293.920,LS,NaN,NaN,NaN,NaN,Z,NaN,NaN,NaN,408329570,1,63,NaN,NaN,"0000,0","000,00","000,00","000,0000","00000,00","0000,00","00000,00","0000,00",NaN,NaN,NaN,NaN,NaN,"00,00",NaN,"00000,00","0000,00",627722.0
253936,LF,231216,1493,1600,LFSB,LSHZ,EC45,RGA02,SAZ,2638,CHE-116.293.920,LS,NaN,NaN,4B43B1,1.0,Z,NaN,NaN,NaN,408537484,1,79,NaN,NaN,"0000,0","000,00","000,00","000,0000","00000,00","0000,00","00000,00","0000,00",NaN,NaN,NaN,NaN,NaN,"00,00",NaN,"00000,00","0000,00",627722.0
253937,LS,231216,544,1720,LSHZ,LFSB,EC45,RGA02,SAZ,2638,CHE-116.293.920,LS,NaN,NaN,NaN,NaN,Z,NaN,NaN,NaN,408377648,1,80,NaN,NaN,"0000,0","000,00","000,00","000,0000","00000,00","0000,00","00000,00","0000,00",NaN,NaN,NaN,NaN,NaN,"00,00",NaN,"00000,00","0000,00",627722.0
253938,LS,231221,271,1211,LSGG,LSHI,EC45,RGA03,SAZ,2638,CHE-116.293.920,LS,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,408571370,1,101,NaN,NaN,"0000,0","000,00","000,00","000,0000","00000,00","0000,00","00000,00","0000,00",NaN,NaN,NaN,NaN,NaN,"00,00",NaN,"00000,00","0000,00",627722.0


In [185]:
# ABACUS = ABACUS.apply(format_flight_message2, axis=1).tolist()
ABACUS = ABACUS.apply(format_flight_message2, axis=1).tolist()

ABACUS = pd.DataFrame(ABACUS)


In [186]:
ABACUS['Time of departure/entry'] = pd.to_numeric(ABACUS['Time of departure/entry'], errors='coerce')

In [187]:
ABACUS.shape[0]

2456

In [188]:
ABACUS[(ABACUS['24-bit Aircraft Address'] == "nan")].shape[0]

0

In [189]:
ABACUS[ABACUS["24-bit Aircraft Address"].isna()].shape[0]

82